#**NN Project**
### Prajvi Saxena
### Md. Jonybul Islam

Code for Task 2 

In [ ]:
# Imports
import pandas as pd
import string 
import numpy as np
import csv
import math
from torch import nn
import torch
from torch.utils.data import DataLoader,TensorDataset
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# upload files 
from google.colab import files
uploaded = files.upload()

# from google.colab import drive
# drive.mount("/content/gdrive")

In [ ]:
# stopword and punctuations removal
data = pd.read_csv('hindi_hatespeech.tsv', sep='\t')
stop_word_list = pd.read_csv('stopword.txt', sep='\s+', header=None)
stop_word_list = stop_word_list[0].tolist()
data['text'] = data['text'].str.replace('[{}]'.format(string.punctuation), '')
data['text'] = data['text'].str.replace('[{}]'.format('।'), '')
data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in (stop_word_list)]))
data['task_1'] = data['task_1'].map({'HOF': 1, 'NOT': 0})

In [ ]:
# getting list of wunique words
V = list(data['text'].str.split(' ', expand=True).stack().unique())
print(len(V))
def word_to_one_hot(word, vocab):
  return vocab[word]

In [ ]:
# getting all words in the corpus
def all_word_corpus(data):
  all_word_list = []
  for value in data.str.split(" "):
    all_word_list.extend(value)
    # for each in value.split(" "):
    #   # print(each)
    # word_list.append(value)
  return all_word_list

all_word_list = all_word_corpus(data["text"])
len(all_word_list)

In [ ]:

# function for Subsampling
def sampling_prob(word, corpus):
  word_number_list = []
  word_freq_dict = {} # Each word frequency dictionary
  word_prob_dict = {}
  for each in word:
    zw_i = corpus.count(each)/ len(corpus)
    word_number_list.append(zw_i)
    word_freq_dict[each] = zw_i
    Pkeepw_i = (math.sqrt(zw_i/0.001) + 1) * (0.001/zw_i)
    word_prob_dict[each] = Pkeepw_i
  return word_freq_dict, word_prob_dict

In [ ]:
word_frequency_dictionary, word_probability_dictionary = sampling_prob(V, all_word_list)


In [ ]:
# New unique word list(V), New Word frquency dictionary, new text data(data)
def new_word_probability_dictionary(word_probability_dictionary, V, all_word_list, data):
  threshold = 1.9
  deleted_word_list = []
  for i in word_probability_dictionary:
    if word_probability_dictionary[i] <= threshold: 
      deleted_word_list.append(i)
  data["text"] = data["text"].apply(lambda x: ' '.join([item for item in x.split() if item not in (deleted_word_list)]))
  V = list(data["text"].str.split(' ', expand=True).stack().unique())
  all_word_list = all_word_corpus(data["text"])
  word_frequency_dictionary, word_probability_dictionary = sampling_prob(V, all_word_list)
  return word_frequency_dictionary, word_probability_dictionary, V, data
# print("word_probability_dictionary:", len(word_probability_dictionary))

In [ ]:
word_frequency_dictionary, word_probability_dictionary, V, data= new_word_probability_dictionary(word_probability_dictionary, V, all_word_list, data)
len(word_probability_dictionary)

In [ ]:
# one hot vocabulary
df = pd.DataFrame(list(zip(V)), columns=['Vocalbulary'])
one_hot_vocabulary = pd.get_dummies(df.Vocalbulary)


In [ ]:
# splitting each sentence of size 25, adding padding of 0's if less than 25.
# to make of same size
for each in range(len(data['text'])):
  if len(data['text'][each].split(" "))<= 25:
    for i in range(25 - len(data['text'][each].split(" "))):
      data['text'][each] = data['text'][each] + " 0"
  elif len(data['text'][each].split(" "))> 25:
    x = data['text'][each].split(" ")[:25]
    data['text'][each] = " ".join(x)
  else:
    pass

In [ ]:
# model structure for word2vec embedding model
class Word2Vec(nn.Module):
  def __init__(self, length, embedding_size):
    super().__init__()
    self.input_layer = nn.Linear(in_features=length, out_features=embedding_size, bias=False)
    self.output = nn.Linear(in_features=embedding_size, out_features=length, bias=False)

  def forward(self, one_hot):
    x=self.input_layer(one_hot)
    x2 = self.output(x)
    out = F.log_softmax(x2)
    return out



In [ ]:
# Previous model load.
model = Word2Vec(20150, 300)
model.load_state_dict(torch.load("model"))
  

In [ ]:
# Set hyperparameters
window_size = 4
embedding_size = 300
learning_rate = 0.001
epochs = 50

In [ ]:
# getting data
class sample_data():

    def __init__(self,data):
      self.data = data
    def __len__(self):
       return len(self.data)        

    def __getitem__(self, idx):

      word_to_one_hot_list_for_tensor = []
      for each in data["text"][idx].split(' '):
        word_to_one_hot_list_for_tensor.append(word_to_one_hot(each,one_hot_vocabulary))

      _label=0.0
      if self.data['task_1'][idx]==1:
        _label=1.0
      return torch.Tensor(word_to_one_hot_list_for_tensor), torch.tensor(_label)



In [ ]:
sample_custom_dataset = sample_data(data)
print(len(sample_custom_dataset))

In [ ]:
# dividing data in 80% training and 20% test
# loading data with batch size 12. 

tr_size = int(0.8 * len(sample_custom_dataset))
# print(tr_size)
te_size = len(sample_custom_dataset) - tr_size

train_dataset, test_dataset = torch.utils.data.random_split(sample_custom_dataset, [tr_size, te_size])
# print(len(train_dataset))

trainloader = DataLoader(train_dataset, batch_size= 12, shuffle = True)
# print(len(trainloader))
testloader = DataLoader(test_dataset, batch_size= 12, shuffle = True)

In [ ]:
# Classifier model definition and forward pass

class Sentiment_Analysis(nn.Module):
  def __init__(self, length, embedding_size):
    super().__init__()
    
    self.layer1 = nn.Linear(in_features=20150*25, out_features=1000, bias=False) 
    self.bn1 = nn.BatchNorm1d(num_features=1000)

    self.layer2 = nn.Linear(in_features=1000, out_features=300, bias=False)
    self.bn2 = nn.BatchNorm1d(num_features=300) 

    self.fc1 = model.input_layer.weight
    self.fc2 = model.output.weight

    self.layer5 = nn.Linear(in_features=embedding_size, out_features=embedding_size, bias=False)
    self.bn3 = nn.BatchNorm1d(num_features=embedding_size)
    self.layer6 = nn.Linear(in_features=embedding_size, out_features=500, bias=False)
    self.bn4 = nn.BatchNorm1d(num_features=500)
    
    self.final = nn.Linear(in_features=500, out_features=1, bias = False)


  def forward(self, one_hot):

    one_hot = one_hot.reshape(len(one_hot), 25*len(V))

    x = torch.relu(self.bn1(self.layer1(one_hot)))
    x = torch.relu(self.bn2(self.layer2(x)))

    x1 = torch.matmul(x, self.fc1)
    x2 = torch.matmul(x1, self.fc2)
    x2 = torch.relu(x2)

    x = torch.relu(self.bn3(self.layer5(x2)))

    x = torch.relu(self.bn4(self.layer6(x)))

    x = (self.final(x))

    out = torch.sigmoid(x)
    return out

sentiment = Sentiment_Analysis(len(V), embedding_size)
torch.cuda.empty_cache()
if torch.cuda.is_available():
    sentiment.cuda()


In [ ]:
# Define Adam optimizer and BCE loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(sentiment.parameters(), lr=learning_rate)
torch.cuda.empty_cache()
criterion = nn.BCELoss()
criterion = criterion.to(device)

In [ ]:
# Train sentiment classifier model for hindi

torch.cuda.empty_cache()
def train(dataset):
  print("Training started")
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  total_loss = 0
  stopping_loss_threshold = 1.0
  min_loss = 10000
  early_stop = False
  epochs_loss = []

  # loop for epochs
  # stopping criteria is using early stopping
  for epoch in range(epochs):
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(dataset):

      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device)

      sentiment.zero_grad()
      # getting predcited output
      log_probs = sentiment(inputs)
      labels = labels.unsqueeze( 1)
      # getting loss
      loss = criterion(log_probs, labels)

      loss.backward()
      optimizer.step()

      running_loss += loss.item() 

    print('epoch: %d, loss: %.3f' %
                  (epoch + 1, running_loss/len(dataset)))
    epochs_loss.append(running_loss/len(dataset))
    # Early stopping
    if (running_loss/len(dataset)) < min_loss:
      min_loss = running_loss/len(dataset)
      model_state_to_save = sentiment.state_dict()
    else:
      print("Early Stopped")
      break

    
       
  print('Minimum loss after early stop:', min_loss)
  plt.plot(epochs_loss, label="Training Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()

  PATH = './modelhindi.pt'
  torch.save(model_state_to_save, PATH)

train(trainloader)

print("Training finished")



In [ ]:
# sentiment = Sentiment_Analysis(len(V), embedding_size)
# torch.cuda.empty_cache()
# if torch.cuda.is_available():
#     sentiment.cuda()

# sentiment.load_state_dict(torch.load("/content/modelhindi.pt"))

In [ ]:
# Testing on hindi dataset
def test(dataset):
    print("Testing started")
    count=0
    
    hitcount=0
    for inputs, labels in dataset: 
      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device)
      
      optimizer.zero_grad()    
      log_probs = sentiment(inputs)

      labelscount=-1
      # getting the count of correct classification(hitcount)
      for i in log_probs:
        count+=1        
        labelscount+=1
        if i <0.5: 
          if torch.eq(labels[labelscount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(labels[labelscount], torch.tensor(1.0)):
            hitcount=hitcount+1
      
      labels = labels.unsqueeze(1)
      loss = criterion(log_probs, labels)

    print('accuracy:', hitcount/count)

test(testloader)

print("Testing finished")

##task2**.2**

In [ ]:
# loading bengali dataset
data = pd.read_csv(r'/content/bengali_hatespeech.csv')

In [ ]:
# Stopword list reader
stop_word_list = pd.read_csv(r'/content/banglastopword.txt', sep='\s+', header=None)
stop_word_list = stop_word_list[0].tolist() 

In [ ]:
# Removing Punctuations and other unnecessary signs
data['sentence'] = data['sentence'].str.replace('[{}]'.format(string.punctuation), '')
data['sentence'] = data['sentence'].str.replace('[{}]'.format('।'), '')
data['sentence'] = data['sentence'].str.replace('[{}]'.format('\n'), '')
data['sentence'] = data['sentence'].str.lower()

In [ ]:
# Removing stopwords
data['sentence'] = data['sentence'].apply(lambda x: ' '.join([item for item in x.split() if item not in (stop_word_list)]))
len(data['sentence'])

In [ ]:
# slicing the dataset to make it of same size as hindi dataset
data1 = data[6000:14280] 
df = data[:7]
data1 = data1.append(df)
data1 = data1.sample(frac=1)
data1 = data1.reset_index()

In [ ]:
if 'index' in data1:
  del data1['index']

if 'category' in data1:
  del data1['category']

In [ ]:
# splitting the sentences with 25 words, if less than 25, we add a padding 
for each in range(len(data['sentence'])):
  if len(data['sentence'][each].split(" "))<= 25:
    for i in range(25 - len(data['sentence'][each].split(" "))):
      data['sentence'][each] = data['sentence'][each] + " জয়"
  elif len(data['sentence'][each].split(" "))> 25:
    x = data['sentence'][each].split(" ")[:25]
    data['sentence'][each] = " ".join(x)
  else:
    pass

In [ ]:
data = data1

In [ ]:
# getting list of unique words
V = list(data['sentence'].str.split(' ', expand=True).stack().unique())
# print(V.index('0'))

In [ ]:
# One hot grabbing function
def word_to_one_hot(word, vocab):
  return vocab[word]


In [ ]:
# all words in the corpus
def all_word_corpus(data):
  all_word_list = []
  for value in data.str.split(" "):
    all_word_list.extend(value)

  return all_word_list

all_word_list = all_word_corpus(data["sentence"])
len(all_word_list)

In [ ]:
# Subsampling function
def sampling_prob(word, corpus):
  word_number_list = []
  word_freq_dict = {} # Each word frequency dictionary
  word_prob_dict = {}
  for each in word:
    zw_i = corpus.count(each)/ len(corpus)
    
    word_number_list.append(zw_i)
    word_freq_dict[each] = zw_i
    Pkeepw_i = (math.sqrt(zw_i/0.001) + 1) * (0.001/zw_i)
    word_prob_dict[each] = Pkeepw_i
  return word_freq_dict, word_prob_dict

In [ ]:
word_frequency_dictionary, word_probability_dictionary = sampling_prob(V, all_word_list)


In [ ]:
# getting word probability dictionary
def new_word_probability_dictionary(word_probability_dictionary, V, all_word_list, data):
  threshold = 1.9
  deleted_word_list = []

  for i in word_probability_dictionary:
    if word_probability_dictionary[i] <= threshold: 
      deleted_word_list.append(i)
  data["sentence"] = data["sentence"].apply(lambda x: ' '.join([item for item in x.split() if item not in (deleted_word_list)]))
  V = list(data["sentence"].str.split(' ', expand=True).stack().unique())
  all_word_list = all_word_corpus(data["sentence"])
  word_frequency_dictionary, word_probability_dictionary = sampling_prob(V, all_word_list)
  return word_frequency_dictionary, word_probability_dictionary, V, data
# print("word_probability_dictionary:", len(word_probability_dictionary))

In [ ]:
word_frequency_dictionary, word_probability_dictionary, V, data= new_word_probability_dictionary(word_probability_dictionary, V, all_word_list, data)
len(word_probability_dictionary)

In [ ]:
df = pd.DataFrame(list(zip(V)), columns=['Vocalbulary'])
one_hot_vocabulary = pd.get_dummies(df.Vocalbulary)

In [ ]:
# skipgram
window_size = 4 
def get_target_context(sentence, V):
  sentence_word_list = []
  
  for each in sentence.split(" "):
    sentence_word_list.append(each)
  
  for i in sentence_word_list:
    sentence_word_index = {}

    # For sentence size more than 4
    if len(sentence_word_list)>= window_size+1:
      # 
      if (len(sentence_word_list) - sentence_word_list.index(i)) > 2:

        if sentence_word_list.index(i)==0:
          sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[4])]

        elif sentence_word_list.index(i)==1:
          sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[4])]
        else:
          index_num = sentence_word_list.index(i)
          sentence_word_index[i] = [V.index(sentence_word_list[index_num-2]),V.index(sentence_word_list[index_num-1]),V.index(sentence_word_list[index_num+1]),V.index(sentence_word_list[index_num+2])]
      elif (len(sentence_word_list) - sentence_word_list.index(i)) == 2:
        sentence_word_index[i] = [V.index(sentence_word_list[len(sentence_word_list)-5]),V.index(sentence_word_list[len(sentence_word_list)-4]),V.index(sentence_word_list[len(sentence_word_list)-3]),V.index(sentence_word_list[len(sentence_word_list)-1])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[len(sentence_word_list)-5]),V.index(sentence_word_list[len(sentence_word_list)-4]),V.index(sentence_word_list[len(sentence_word_list)-3]),V.index(sentence_word_list[len(sentence_word_list)-2])]
      yield sentence_word_index

    # For sentence size 4
    elif len(sentence_word_list) == window_size:
      if sentence_word_list.index(i) == 0:
      
        sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[1])]
     
      elif sentence_word_list.index(i)==1:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[0])]
      elif sentence_word_list.index(i)==2: 
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[1]),V.index(sentence_word_list[3]),V.index(sentence_word_list[0])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[0])]
      yield sentence_word_index

    # For sentence size 3
    elif len(sentence_word_list) == window_size-1:
      if sentence_word_list.index(i) == 0:
        sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[1]),V.index(sentence_word_list[2])]
      elif sentence_word_list.index(i)==1:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[2]),V.index(sentence_word_list[0]),V.index(sentence_word_list[2])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[1]),V.index(sentence_word_list[0]),V.index(sentence_word_list[1])]
      yield sentence_word_index

    # For sentence size 2
    elif len(sentence_word_list) == window_size-2:
      if sentence_word_list.index(i) == 0:
        sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[1]),V.index(sentence_word_list[1]),V.index(sentence_word_list[1])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0])]
      yield sentence_word_index

    # For sentence size 1
    elif len(sentence_word_list) == window_size-3:
      sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0])]
      yield sentence_word_index

    else:
      pass
    

In [ ]:
# All (current_word, context) dictionary
all_yield_wrods_context = {}
for each in data['sentence']:
  # print(each)
  gt = get_target_context(each, V)
  for x in gt:
    all_yield_wrods_context.update(x)

len(all_yield_wrods_context)

##2.3

In [ ]:
# Set hyperparameters
window_size = 4
embedding_size = 300
learning_rate = 0.05
epochs = 50

In [ ]:
# DataLoader for embeddings
def dataloader_for_model(all_yield_wrods_context, V):
  word_to_one_hot_list_for_tensor = []
  context_list_for_tensor = []
  df = pd.DataFrame(list(zip(V)), columns=['Vocalbulary'])
  one_hot_vocabulary = pd.get_dummies(df.Vocalbulary)
  for i in all_yield_wrods_context:
    word_to_one_hot_list_for_tensor.append(word_to_one_hot(i,one_hot_vocabulary))
    context_list_for_tensor.append(all_yield_wrods_context[i])

  one_hot_tensor = torch.FloatTensor(word_to_one_hot_list_for_tensor)
  context_tensor = torch.FloatTensor(context_list_for_tensor)
  # print(torch.max(context_tensor,1)[1])
  dataset = TensorDataset(one_hot_tensor,context_tensor)


  loader = DataLoader(
      dataset,
      batch_size=32,
      num_workers=0,
      shuffle=True
  )
  return loader
  

In [ ]:
# word2vec embedding model definition and forward function
class Word2Vec(nn.Module):
  def __init__(self, length, embedding_size):
    super().__init__()
    self.input_layer = nn.Linear(in_features=length, out_features=embedding_size, bias=False)
    self.output = nn.Linear(in_features=embedding_size, out_features=length, bias=False)

  def forward(self, one_hot):
    x=self.input_layer(one_hot)
    x = self.output(x)
    
    out = F.log_softmax(x)
    return out

torch.cuda.empty_cache()
model = Word2Vec(len(word_probability_dictionary), embedding_size)
if torch.cuda.is_available():
    model.cuda()


In [ ]:
# Previous model load.
model = Word2Vec(20150, 300)
model.load_state_dict(torch.load("model"))

if torch.cuda.is_available():
    model.cuda() 

In [ ]:
# Define Adam optimizer and NLL loss
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
criterion = criterion.to(device)

In [ ]:
# Define train procedure for word2vec embeddings
losses = []

def train(dataset):
  print("Training started")
  
  total_loss = 0
  min_loss = 10000
  stopping_loss_threshold = 1.0
  early_stop = False
  epochs_loss = []

  for epoch in range(epochs):
    running_loss = 0.0

    for i, data in enumerate(dataset):
      inputs, labels = data
      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device, dtype=torch.int64)

      model.zero_grad()
      log_probs = model(inputs)
      loss = criterion(log_probs, torch.max(labels, 1)[1])
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

      #early stopping
      if i % 100 == 99:    # print every 100 mini-batches
        print('epoch:%d, batch:%d, loss: %.3f' %
          (epoch + 1, i + 1, total_loss / 100))
        if (total_loss /100 ) <=stopping_loss_threshold:
          print("Early Stop.")
          print('epoch:%d, loss: %.3f' %
              (epoch + 1, total_loss / 100))
          early_stop = True
          break
        total_loss = 0.0

    if early_stop:
      print("Stopped")
      break
   
train(dataloader_for_model(all_yield_wrods_context, V))

print("Training finished")

In [ ]:
# Save the model
PATH = './modelbengali.pt'
torch.save(model.state_dict(), PATH)

##2.4

In [ ]:
#sample function for bangali dataset
class sample_data():
    def __init__(self,data):
      self.data = data
    def __len__(self):
       return len(self.data)        

    def __getitem__(self, idx):

      word_to_one_hot_list_for_tensor = []
      for each in data["sentence"][idx].split(' '):
        for i in V:
          if i in each and each in i:
            word_to_one_hot_list_for_tensor.append(1)
          else:
            word_to_one_hot_list_for_tensor.append(0)

      _label=0.0
      if self.data['hate'][idx]==1:
        _label=1.0
      return torch.Tensor(word_to_one_hot_list_for_tensor), torch.tensor(_label)


In [ ]:
sample_custom_dataset = sample_data(data)

In [ ]:
# dividing dataset to 80% training and 20% test set
# dataloader with batch size 32
tr_size = int(0.8 * len(sample_custom_dataset))
te_size = len(sample_custom_dataset) - tr_size

train_dataset, test_dataset = torch.utils.data.random_split(sample_custom_dataset, [tr_size, te_size])

trainloader = DataLoader(train_dataset, batch_size= 32, shuffle = True)
testloader = DataLoader(test_dataset, batch_size= 32, shuffle = True)

In [ ]:
# loading the classifier model trained for hindi dataset in task 2.1
sentiment.load_state_dict(torch.load("/content/modelhindi.pt"))

In [ ]:
# Define Adam optimizer and BCE loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(sentiment.parameters(), lr=learning_rate)

criterion = nn.BCELoss()
criterion = criterion.to(device)

In [ ]:
# testing bengali dataset on classifier trained on hindi dataset(2.1)
def test(dataset):
    print("Testing started")
    count=0

    hitcount=0
    for inputs, labels in dataset: 

      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device)
      
      # 1. Generate predictions
      optimizer.zero_grad()
      log_probs = sentiment(inputs)

      # increasing hitcount when a correct prediction is made
      labelscount=-1
      for i in log_probs:
        count+=1        
        labelscount+=1
        if i <0.5:
          if torch.eq(labels[labelscount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(labels[labelscount], torch.tensor(1.0)):
            hitcount=hitcount+1

      # 2. Calculate loss
      labels = labels.unsqueeze(1)
      loss = criterion(log_probs, labels)

    print('accuracy:', hitcount/count)

test(testloader)

print("Testing finished")

In [ ]:
# re-training of the already trained classifier model for hindi dataset(2.1) now with bengali dataset
# transfer learning

def train(dataset):
  print("Training started")
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  total_loss = 0
  stopping_loss_threshold = 1.0
  min_loss = 10000
  early_stop = False
  epochs_loss = []

  for epoch in range(epochs):
    print(len(dataset))
    running_loss = 0.0
    counter = 0
    
    for i, (inputs, labels) in enumerate(dataset):
      counter += 1
      
      inputs, labels = inputs.cuda(), labels.cuda()

      sentiment.zero_grad()
      log_probs = sentiment(inputs)

      labels = labels.unsqueeze( 1)
      loss = criterion(log_probs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() 
      # total_loss += loss.item()

    # Early stopping as termination condition
    print('epoch: %d, loss: %.3f' %
                  (epoch + 1, running_loss/len(dataset)))
    epochs_loss.append(running_loss/len(dataset))
    if (running_loss/len(dataset)) < min_loss:
      min_loss = running_loss/len(dataset)
      model_state_to_save = sentiment.state_dict()
    else:
      print("Early Stopped")
      break
            
  print('Minimum loss after early stopping is:', min_loss)
  plt.plot(epochs_loss, label="Training Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()

  #saving the model
  PATH = './combine.pt'
  torch.save(model_state_to_save, PATH)

   
train(trainloader)

print("Training finished")



In [ ]:
# testing the retrained classifier model for bengali test dataset
def test(dataset):
    print("Testing started")
    count=0

    hitcount=0
    for inputs, labels in dataset: 

      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device)
      
      
      # 1. Generate predictions
      optimizer.zero_grad()
    
      log_probs = sentiment(inputs)

      labelscount=-1
      for i in log_probs:
        count+=1        
        labelscount+=1
        if i <0.5:
          if torch.eq(labels[labelscount], torch.tensor(0.0)):
            hitcount=hitcount+1
        elif i>=.5:
          if torch.eq(labels[labelscount], torch.tensor(1.0)):
            hitcount=hitcount+1

      
      # Calculate loss
      labels = labels.unsqueeze(1)
      loss = criterion(log_probs, labels)

    print('accuracy:', hitcount/count)


test(testloader)

print("Testing finished")

Thank You. 
Code for part 3 is in seperate ipynb file